In [1]:
import pandas as pd

In [11]:
df = pd.read_excel("../../../data/10_1371_journal_pone_0289097/pone.0289097.s001.xlsx", sheet_name="Sheet1")

In [12]:
df

,id,dateofinterview,age,numberofchildren,ageatfirstbirth,ageyoungest,attendedformaleducation,educationlvl,electricity,radio,...,howoftendoyoulistentoradio,howoftendoyouwatchtelevision,massageaboutmalariapreventiontre,whatwherethemassagesabout,perceivedsusceptibility,perceivedseverity,susceptibility,severity,cat_susceptbility,cat_severity
0,1,2015-01-21,22,3,3,3,1,2.0,1,1,...,0,4,1,1.0,14,1,14.0,1,LOW,LOW
1,2,2015-01-21,21,1,7,0,1,2.0,0,1,...,4,0,1,8.0,16,2,16.0,2,LOW,HIGH
2,3,2015-01-20,26,3,6,3,1,4.0,0,1,...,4,0,1,8.0,14,1,14.0,1,LOW,LOW
3,4,2015-01-16,31,5,2,2,1,2.0,0,0,...,0,0,1,4.0,20,1,20.0,1,HIGH,LOW
4,5,2015-01-21,18,1,5,0,1,4.0,1,0,...,1,3,1,8.0,17,1,17.0,1,HIGH,LOW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,226,2015-01-12,26,4,5,1,1,NaN,0,0,...,2,0,0,NaN,17,1,17.0,1,HIGH,LOW
226,227,2015-01-12,33,7,3,2,1,NaN,0,1,...,4,0,1,1.0,15,1,15.0,1,LOW,LOW
227,228,2015-01-12,19,1,6,0,1,NaN,1,1,...,4,0,1,1.0,14,2,NaN,2,NaN,HIGH
228,229,2015-01-19,20,1,7,0,1,NaN,1,1,...,4,4,1,1.0,18,3,18.0,3,HIGH,HIGH


## Deriving susceptibility, severity

In [13]:
df = df[~(df.q301.isna() | df.q402.isna())].reset_index(drop=True)

In [14]:
df["perceived_susceptibility"] = df.q301.map(lambda x : True if x <= 2 else False)
df["perceived_severity"]       = df.q402.map(lambda x : True if x <= 2 else False)

$p(\text{behaviour}\mid\text{perceived susceptibility, severity}=\text{low})=p(\text{ITN use}=1\mid\text{perceived susceptibility, severity}=\text{low})$

In [34]:
df[df.perceived_susceptibility & df.perceived_severity].howoftenusemosquitonet.mean()

1.9536423841059603

In [35]:
df[~(df.perceived_susceptibility | df.perceived_severity)].howoftenusemosquitonet.mean()

1.4285714285714286

In [31]:
(len(df[df.perceived_susceptibility & df.perceived_severity].howoftenusemosquitonet),
len(df[~(df.perceived_susceptibility | df.perceived_severity)].howoftenusemosquitonet))

(152, 7)

In [8]:
df = df[~(df.cat_susceptbility.isna() | df.cat_severity.isna())].reset_index(drop=True)

In [18]:
susc = df.groupby(by=['sleepinmosquitonetlastnight', 'cat_susceptbility']).id.count()
susc

sleepinmosquitonetlastnight  cat_susceptbility
0                            HIGH                 21
                             LOW                  19
1                            HIGH                 67
                             LOW                  72
Name: id, dtype: int64

In [19]:
sev = df.groupby(by=['sleepinmosquitonetlastnight', 'cat_severity']).id.count()
sev

sleepinmosquitonetlastnight  cat_severity
0                            HIGH            15
                             LOW             25
1                            HIGH            43
                             LOW             96
Name: id, dtype: int64

$p(\text{behaviour}\mid\text{perceived susceptibility, severity}=\text{low})=p(\text{ITN use}=1\mid\text{perceived susceptibility, severity}=\text{low})$

Derive frequent use as '1' or '2' in "how often do you use?" => 3–5, consider infrequent

In [49]:
df["freq_use"] = df.howoftenusemosquitonet.map(lambda x : 1 if (x > 0 and x < 3) else 0)

In [50]:
low_both = df[(df.cat_susceptbility == "LOW") & (df.cat_severity == "LOW")]
low_both.freq_use.sum()/len(low_both)

0.7727272727272727

In [51]:
low_both = df[(df.cat_susceptbility == "HIGH") & (df.cat_severity == "HIGH")]
low_both.freq_use.sum()/len(low_both)

0.7272727272727273

In [33]:
p = ((df.cat_susceptbility == "LOW") & (df.cat_severity == "LOW")).sum()/len(df)
print(f"p:\t{p}\nOR:\t{p/(1-p)}")

p:	0.3687150837988827
OR:	0.5840707964601769
